In [1]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install(version = "3.10")
# BiocManager::install("randomForest")

In [2]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Input')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Smoking Status Prediction/Output')
cur_date = "021822"

library(tidyverse)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(pROC)
library(randomForest)

#reading in file
molecular_feature_df_w_Ecig_40 = read_csv("20220217_mf_qrilc_w40.csv") %>%
     rename(Smoking_Status = sample)
molecular_feature_df_wo_Ecig_40 = read_csv("20220217_mf_qrilc_wo40.csv") %>%
     rename(Smoking_Status = sample)
# ("021422_imputed_cytokine_data_w_Ecig40.csv") %>%
#     rename(Smoking_Status = X1)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [3]:
head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS_110,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS_143,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS_152,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS_187,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS_196,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


In [4]:
#first making smoking status column binary denoted by 0 = non smoker and 1 = ecig user
molecular_feature_df_w_Ecig_40 = molecular_feature_df_w_Ecig_40 %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

molecular_feature_df_wo_Ecig_40 = molecular_feature_df_wo_Ecig_40 %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
0,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
0,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
0,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
0,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
0,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


The goal of this analysis is to use molecular features [continuous] to predict smoking staus (dichotomous). This will be done using Support Vector Machine (SVM) based on a logistic regression model for classification or Random Forest.

# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [5]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

#calling fn
#linear kernel
svm_linear_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
svm_linear_wo_Ecig_40 = svm_classification(molecular_feature_df_wo_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
svm_radial_wo_Ecig_40 = svm_classification(molecular_feature_df_wo_Ecig_40, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)
svm_polynomial_wo_Ecig_40 = svm_classification(molecular_feature_df_wo_Ecig_40, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [13]:
#creating one combined df
svm_final_df = data.frame(rbind(svm_linear_w_Ecig_40, svm_radial_w_Ecig_40, svm_polynomial_w_Ecig_40,
                               svm_linear_wo_Ecig_40, svm_radial_wo_Ecig_40, svm_polynomial_wo_Ecig_40))

svm_final_df = data.frame(Dataset = c(rep(c("Ecig_40"), times = 3), rep(c("no_Ecig_40"), times = 3)), 
                          Kernel = rep(c("linear", "radial", "polynomial"), times = 2), svm_final_df)
svm_final_df

Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_40,linear,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5424242,0.5000000
Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6187013,0.6057143
no_Ecig_40,linear,0.8100000,0.8000000,0.8200000,0.8666667,0.8100000
no_Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5348485,0.5000000
no_Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6070996,0.6057143


> The radial kernel performed the best overall. 

Determing if there are highly correlated variables that can be removed, because multicollinearity makes variables that should be significant appear insignificant.

In [7]:
#creating correlation matrix
#the cut off can be arbitrary, but due to the high number of correlations, 0.85 became the cutoff
corr_matrix = rcorr(as.matrix(molecular_feature_df[2:29]), type = "spearman")
#corr_matrix = rcorr(as.matrix(molecular_feature_df[2:3051]), type = "spearman")
#creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df = data.frame(corr_matrix$r) %>%
  rownames_to_column(var = "Variable1") %>%
  #needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  #also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  #removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df)

Using Variable1 as id variables



,Variable1,Variable2,Spearman Coefficient
,<chr>,<fct>,<dbl>
1,IL2,IL10,0.9729682
2,TNFa,IL13,0.9715371
3,IL2,IFNg,0.9636903
4,IL12p70,IFNg,0.9484512
5,IL10,IFNg,0.9451336
6,IL2,IL12p70,0.9435556


In [8]:
#determining how many times each feature occurs
correlation_occurences1 = corr_matrix_df %>%
    group_by(Variable1) %>%
    count() %>%
    arrange(-n) %>%
    filter(n > 20)

#correlation_occurences1

correlation_occurences2 = corr_matrix_df %>%
    group_by(Variable2) %>%
    count() %>%
    arrange(-n) %>%
    filter(n > 20)

#correlation_occurences2

all_corr_occurences = unique(c(correlation_occurences1$Variable1, as.character(correlation_occurences2$Variable2)))

In [9]:
all_corr_occurences

character(0)

In [10]:
#removing molecular features (MF) that correlate most often
`%notin%` <- Negate(`%in%`)
penalized_dataset = molecular_feature_df[,colnames(molecular_feature_df) %notin% all_corr_occurences]

In [13]:
#calling fn
#linear kernel
svm_linear = svm_classification(penalized_dataset, "linear", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial = svm_classification(penalized_dataset, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial = svm_classification(penalized_dataset, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)


#creating one combined df
svm_penalized_final_df = data.frame(rbind(svm_linear, svm_radial, svm_polynomial))
svm_penalized_final_df = data.frame(Kernel = c("linear", "radial", "polynomial"), svm_penalized_final_df)
svm_penalized_final_df

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.4804762,0.5476190,0.4133333,0.5018254,0.5766667
radial,0.5252381,0.4904762,0.5600000,0.6000000,0.5919048
polynomial,0.4528571,0.4523810,0.4533333,NaN,0.4528571


In [15]:
svm_final_df
svm_penalized_final_df

Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.4804762,0.5476190,0.4133333,0.5018254,0.5766667
radial,0.5252381,0.4904762,0.5600000,0.6000000,0.5919048
polynomial,0.4528571,0.4523810,0.4533333,NaN,0.4528571


Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.4804762,0.5476190,0.4133333,0.5018254,0.5766667
radial,0.5252381,0.4904762,0.5600000,0.6000000,0.5919048
polynomial,0.4528571,0.4523810,0.4533333,NaN,0.4528571


# Random Forest

- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [ ]:
rf_classification = function(dataset, outcome, pred_outcome){
    #setting for reproducibility
    set.seed(12)
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) #number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 #number of variables in dataset
        mtry_values = c(p/2, sqrt(p), p)

        #will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                #print(reg_rf_pred_tune[[k]])
                #print(reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        #viewing OOB errors
#         print(rf_error_df)

        #finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))
#         print(best_oob_errors)
#         print(rf_error_df$Tree.Number[min(best_oob_errors)])
        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
        #need to go back and figure out how the number of trees/ variables make a difference even though
        #the errors are the same!!!!
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)], 
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        #predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")


        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        #extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "MF")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
    }
    
    #taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(MF) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        #sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    #return training set, matrix and variable importance values
    return(list(data_train, metrics, variable_importance_df))

}

#calling fn
rf_values_w_Ecig_40 = rf_classification(molecular_feature_df_w_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
rf_values_wo_Ecig_40 = rf_classification(molecular_feature_df_wo_Ecig_40, "Smoking_Status", "pred_Smoking_Status")

In [20]:
#viewing results
rf_confusion_matrix = data.frame(Dataset = c("Ecig_40", "no_Ecig_40"), rbind(rf_values_w_Ecig_40[[2]], 
                                                                             rf_values_wo_Ecig_40[[2]]))
rf_confusion_matrix

most_sig_variables_w_Ecig_40 = rf_values_w_Ecig_40[[3]] %>%
    filter(MeanDecreaseGini > 0.05)
most_sig_variables_wo_Ecig_40 = rf_values_wo_Ecig_40[[3]] %>%
    filter(MeanDecreaseGini > 0.05)
most_sig_variables_w_Ecig_40
most_sig_variables_wo_Ecig_40

Dataset,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_40,0.9047619,0.8428571,0.9666667,0.9600000,0.9047619
no_Ecig_40,0.9047619,0.8428571,0.9666667,0.9666667,0.9047619


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.05424062
MF_5,4.68378298
MF_292,0.27705811
MF_3345,0.24800949
MF_1713,0.18760701
MF_4803,0.16542196
MF_7423,0.15562403
MF_5782,0.15079241
MF_7221,0.14144676


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.05972731
MF_5,4.08153303
MF_13899,0.29520675
MF_4858,0.19183248
MF_5547,0.19155348
MF_292,0.18847797
MF_17380,0.15940730
MF_108,0.15200375
MF_873,0.15093885


In [45]:
#now rerunning rf model w/ noise variables to determine which variables rank higher than the noise
train_vars_noise = rf_values[[1]]

# Add random noise predictors as an additional method to evaluate model performance
# Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[3])]], replace = TRUE) 
train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[4])]], replace = TRUE)
train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[7])]], replace = TRUE)

head(train_vars_noise)

Smoking_Status,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,TNFa,⋯,MCP4,MDC,MIP1a,MIP1b,TARC,noise1,noise2,noise3,noise4,noise5
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,⋯,10.072081,31.69299,22.02667,46.80070,20.541303,2.2425790,2.5279121,13.32276,22.83134,4.1920913
1,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,⋯,1.090400,167.32860,46.98751,101.25635,35.541708,0.3368341,2.5279121,48.62919,538.03828,0.8714269
1,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,⋯,21.200301,196.12405,32.89026,24.92311,19.708475,0.6350398,1.2665986,35.34034,39.36707,1.8440114
1,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,⋯,7.189953,139.46181,16.70862,21.35656,9.885379,2.6604979,0.6558602,10.99028,43.89248,3.6939698
1,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,⋯,3.359783,69.35689,9.56433,15.67118,9.302557,0.2135926,4.0799843,60.41427,29.30701,0.7766565
1,2.2133006,1.2942145,0.7038064,34.60048,53.345004,1.8460707,0.33927947,4.781624,2.5971519,⋯,1.464661,98.43296,23.87220,97.01791,36.198486,0.3994781,2.0873758,36.81471,55.43364,0.7766565


In [47]:
#calling rf fn
noise_rf_values = rf_classification(train_vars_noise, "Smoking_Status", "pred_Smoking_Status")
noise_rf_values[2:3]

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases



[[1]]
  Balanced Accuracy Sensitivity Specificity  PPV   AUC
1             0.425        0.46        0.39 0.48 0.615

[[2]]
# A tibble: 34 x 2
   Cytokine MeanDecreaseGini
   <chr>               <dbl>
 1 IL1a                1.53 
 2 noise5              1.00 
 3 MCP10               0.958
 4 TARC                0.889
 5 MIP1a               0.767
 6 noise3              0.714
 7 MDC                 0.706
 8 GMCSF               0.691
 9 VEGF                0.653
10 MCP4                0.611
# … with 24 more rows

In [59]:
noise_importance_values = noise_rf_values[[3]] %>%
    column_to_rownames(var = "Cytokine")

#calculating avg noise rank out of all variables
noise1rank = which(rownames(noise_importance_values) == "noise1")/nrow(noise_importance_values)
noise2rank = which(rownames(noise_importance_values) == "noise2")/nrow(noise_importance_values)
noise3rank = which(rownames(noise_importance_values) == "noise3")/nrow(noise_importance_values)
noise4rank = which(rownames(noise_importance_values) == "noise4")/nrow(noise_importance_values)
noise5rank = which(rownames(noise_importance_values) == "noise5")/nrow(noise_importance_values)
# noise1rank
# noise2rank
# noise3rank
# noise4rank
# noise5rank

#calculating avg mean decrease gini or background noise importance value
avg_noise_rank = (noise1rank + noise2rank + noise3rank + noise4rank + noise5rank)/5
avg_noise_rank

[1] 0.3470588